In [ ]:
#@title 🔬 SHA-256 BACKDOOR DISCOVERY ENGINE - Automatic Search

"""
SHA-256 BACKDOOR DISCOVERY ENGINE
==================================

PHILOSOPHY: Don't test predefined patterns - DISCOVER them!

This tool uses evolutionary search to find ANY combination of
conditions on message A that creates collision paths.

APPROACH:
1. Represent "backdoor conditions" as a genome (which bits to constrain)
2. Evolve population of potential backdoors
3. Fitness = collision quality (matching hash bits)
4. Crossover + Mutation to explore condition space
5. Quaternion heuristics to guide SAT solving

If NSA has a sophisticated backdoor, it's some condition C such that:
  C(A) → SHA256(A) = SHA256(B) for some B ≠ A

We're searching for C automatically.
"""

!pip install python-sat -q

import numpy as np
import hashlib
import time
import random
import json
from collections import defaultdict
from pysat.solvers import Solver as PySolver
import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle
import os

print("="*70)
print("  🔬 SHA-256 BACKDOOR DISCOVERY ENGINE")
print("  Automatic Search for Hidden Weaknesses")
print("="*70)

# ============================================================
# SHA-256 CONSTANTS
# ============================================================

K_CONSTANTS = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

H_INIT = [
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
]


# ============================================================
# QUATERNION ENGINE
# ============================================================

class Quaternion:
    __slots__ = ['q']

    def __init__(self, w=0.0, x=0.0, y=0.0, z=0.0):
        self.q = np.array([w, x, y, z], dtype=np.float64)
        self._stabilize()

    def _stabilize(self):
        self.q = np.clip(self.q, -1e15, 1e15)
        self.q = np.nan_to_num(self.q, nan=0.0, posinf=1e10, neginf=-1e10)
        max_val = np.max(np.abs(self.q))
        if max_val > 1e8:
            self.q /= max_val

    def __add__(self, other):
        r = Quaternion()
        r.q = self.q + other.q
        r._stabilize()
        return r

    def __mul__(self, other):
        if isinstance(other, (int, float)):
            r = Quaternion()
            r.q = self.q * other
            r._stabilize()
            return r
        a, b, c, d = self.q
        e, f, g, h = other.q
        r = Quaternion()
        r.q = np.array([
            a*e - b*f - c*g - d*h,
            a*f + b*e + c*h - d*g,
            a*g - b*h + c*e + d*f,
            a*h + b*g - c*f + d*e
        ], dtype=np.float64)
        r._stabilize()
        return r

    def norm(self):
        return np.sqrt(np.sum(self.q ** 2))

    def to_matrix(self):
        w, x, y, z = self.q
        return np.array([
            [w, -x, -y, -z],
            [x,  w, -z,  y],
            [y,  z,  w, -x],
            [z, -y,  x,  w]
        ], dtype=np.float64)


# ============================================================
# BACKDOOR GENOME
# ============================================================

class BackdoorGenome:
    """
    Represents a potential backdoor condition as a genome.

    A backdoor condition specifies:
    1. Which message bits are constrained (mask)
    2. What value they must have (values)
    3. Which differential bits are active (diff_mask)
    4. Relationships between bits (correlations)
    """

    def __init__(self, genome_size=512):
        self.genome_size = genome_size

        # Condition on message A: which bits to constrain
        # 0 = free, 1 = force to 0, 2 = force to 1
        self.msg_constraints = np.zeros(genome_size, dtype=np.int8)

        # Condition on differential: which bits must differ
        # 0 = free, 1 = must be 0, 2 = must be 1
        self.diff_constraints = np.zeros(genome_size, dtype=np.int8)

        # Bit correlations: pairs of bits that must be equal/opposite
        # List of (bit_i, bit_j, relation) where relation: 0=equal, 1=opposite
        self.correlations = []

        # Fitness metrics
        self.fitness = 0
        self.matching_bits = 0
        self.is_collision = False
        self.msg_A = None
        self.msg_B = None
        self.hash_A = None
        self.hash_B = None

        # Metadata
        self.generation = 0
        self.parent_fitness = 0

    def randomize(self, constraint_density=0.05, diff_density=0.01, corr_count=0):
        """Create random initial genome"""
        # Message constraints: sparse random
        for i in range(self.genome_size):
            if random.random() < constraint_density:
                self.msg_constraints[i] = random.choice([1, 2])

        # Differential: very sparse (usually just 1-3 bits)
        num_diffs = max(1, int(self.genome_size * diff_density))
        diff_positions = random.sample(range(self.genome_size), min(num_diffs, 10))
        for pos in diff_positions:
            self.diff_constraints[pos] = 2  # Must be 1 (different)

        # Correlations
        for _ in range(corr_count):
            i = random.randint(0, self.genome_size - 1)
            j = random.randint(0, self.genome_size - 1)
            if i != j:
                self.correlations.append((i, j, random.choice([0, 1])))

    def mutate(self, mutation_rate=0.1, generation=0):
        """Mutate genome"""
        child = BackdoorGenome(self.genome_size)
        child.msg_constraints = self.msg_constraints.copy()
        child.diff_constraints = self.diff_constraints.copy()
        child.correlations = self.correlations.copy()
        child.generation = generation
        child.parent_fitness = self.fitness

        # Adaptive mutation rate based on fitness
        adaptive_rate = mutation_rate * (1.0 + 0.5 * (256 - self.matching_bits) / 256)

        # Mutate message constraints
        for i in range(self.genome_size):
            if random.random() < adaptive_rate:
                if self.msg_constraints[i] == 0:
                    child.msg_constraints[i] = random.choice([1, 2])
                elif random.random() < 0.3:
                    child.msg_constraints[i] = 0  # Remove constraint
                else:
                    child.msg_constraints[i] = 3 - self.msg_constraints[i]  # Flip 1↔2

        # Mutate differential constraints
        for i in range(self.genome_size):
            if random.random() < adaptive_rate * 0.5:
                if self.diff_constraints[i] == 0:
                    if random.random() < 0.1:  # Rarely add new diff
                        child.diff_constraints[i] = 2
                else:
                    if random.random() < 0.3:
                        child.diff_constraints[i] = 0  # Remove

        # Mutate correlations
        new_corrs = []
        for corr in child.correlations:
            if random.random() > adaptive_rate * 0.2:
                new_corrs.append(corr)

        if random.random() < adaptive_rate:
            i = random.randint(0, self.genome_size - 1)
            j = random.randint(0, self.genome_size - 1)
            if i != j:
                new_corrs.append((i, j, random.choice([0, 1])))

        child.correlations = new_corrs

        return child

    def crossover(self, other, generation=0):
        """Crossover with another genome"""
        child = BackdoorGenome(self.genome_size)
        child.generation = generation
        child.parent_fitness = max(self.fitness, other.fitness)

        # Two-point crossover for constraints
        p1 = random.randint(0, self.genome_size - 1)
        p2 = random.randint(p1, self.genome_size)

        child.msg_constraints[:p1] = self.msg_constraints[:p1]
        child.msg_constraints[p1:p2] = other.msg_constraints[p1:p2]
        child.msg_constraints[p2:] = self.msg_constraints[p2:]

        # Differential: take from fitter parent
        if self.fitness >= other.fitness:
            child.diff_constraints = self.diff_constraints.copy()
        else:
            child.diff_constraints = other.diff_constraints.copy()

        # Correlations: merge
        child.correlations = self.correlations[:len(self.correlations)//2] + \
                            other.correlations[len(other.correlations)//2:]

        return child

    def get_constraint_summary(self):
        """Human-readable summary"""
        msg_zeros = np.sum(self.msg_constraints == 1)
        msg_ones = np.sum(self.msg_constraints == 2)
        diff_active = np.sum(self.diff_constraints == 2)

        return {
            'msg_zeros': int(msg_zeros),
            'msg_ones': int(msg_ones),
            'total_msg_constraints': int(msg_zeros + msg_ones),
            'diff_bits': int(diff_active),
            'correlations': len(self.correlations),
            'fitness': self.fitness,
            'matching_bits': self.matching_bits
        }

    def to_dict(self):
        """Serialize to dict"""
        return {
            'msg_constraints': self.msg_constraints.tolist(),
            'diff_constraints': self.diff_constraints.tolist(),
            'correlations': self.correlations,
            'fitness': self.fitness,
            'matching_bits': self.matching_bits,
            'is_collision': self.is_collision,
            'msg_A': self.msg_A.hex() if self.msg_A else None,
            'msg_B': self.msg_B.hex() if self.msg_B else None,
            'hash_A': self.hash_A if self.hash_A else None,
            'hash_B': self.hash_B if self.hash_B else None
        }


# ============================================================
# FAST SHA-256 DIFFERENTIAL TESTER
# ============================================================

def sha256_differential_test(msg_constraints, diff_constraints, correlations, num_tries=100):
    """
    Fast test of a backdoor condition.

    1. Generate message A satisfying constraints
    2. Generate message B = A XOR diff
    3. Hash both and count matching bits
    4. Return best result
    """
    best_matching = 0
    best_A = None
    best_B = None
    best_hash_A = None
    best_hash_B = None

    for _ in range(num_tries):
        # Generate A satisfying constraints
        A_bits = []
        for i in range(512):
            if msg_constraints[i] == 1:
                A_bits.append(0)
            elif msg_constraints[i] == 2:
                A_bits.append(1)
            else:
                A_bits.append(random.randint(0, 1))

        # Apply correlations
        for (i, j, rel) in correlations:
            if i < 512 and j < 512:
                if rel == 0:  # Equal
                    A_bits[j] = A_bits[i]
                else:  # Opposite
                    A_bits[j] = 1 - A_bits[i]

        # Generate diff
        diff_bits = []
        for i in range(512):
            if diff_constraints[i] == 1:
                diff_bits.append(0)
            elif diff_constraints[i] == 2:
                diff_bits.append(1)
            else:
                diff_bits.append(0)  # Default no diff

        # Ensure at least one diff
        if sum(diff_bits) == 0:
            diff_bits[random.randint(0, 511)] = 1

        # B = A XOR diff
        B_bits = [(A_bits[i] ^ diff_bits[i]) for i in range(512)]

        # Convert to bytes
        def bits_to_bytes(bits):
            words = []
            for w in range(16):
                val = sum((bits[w*32 + b] << b) for b in range(32))
                words.append(val)
            return b''.join(w.to_bytes(4, 'big') for w in words)

        msg_A = bits_to_bytes(A_bits)
        msg_B = bits_to_bytes(B_bits)

        # Hash
        hash_A = hashlib.sha256(msg_A).digest()
        hash_B = hashlib.sha256(msg_B).digest()

        # Count matching bits
        matching = 0
        for a, b in zip(hash_A, hash_B):
            xor = a ^ b
            matching += 8 - bin(xor).count('1')

        if matching > best_matching:
            best_matching = matching
            best_A = msg_A
            best_B = msg_B
            best_hash_A = hash_A.hex()
            best_hash_B = hash_B.hex()

        if matching == 256:
            break

    return best_matching, best_A, best_B, best_hash_A, best_hash_B


# ============================================================
# SAT-BASED CONDITION OPTIMIZER
# ============================================================

class SATConditionOptimizer:
    """
    Uses SAT solving to find optimal message given constraints.
    More powerful than random sampling for complex conditions.
    """

    def __init__(self, num_rounds=64):
        self.num_rounds = num_rounds

    def optimize(self, genome, timeout=30):
        """
        Build SAT problem for the genome's conditions and solve.
        Returns best (matching_bits, msg_A, msg_B).
        """
        # For efficiency, first try random sampling
        matching, msg_A, msg_B, hash_A, hash_B = sha256_differential_test(
            genome.msg_constraints,
            genome.diff_constraints,
            genome.correlations,
            num_tries=50
        )

        # If already good, don't bother with SAT
        if matching >= 200:
            return matching, msg_A, msg_B, hash_A, hash_B

        # For complex conditions, could add SAT-based optimization here
        # For now, just do more random tries
        matching2, msg_A2, msg_B2, hash_A2, hash_B2 = sha256_differential_test(
            genome.msg_constraints,
            genome.diff_constraints,
            genome.correlations,
            num_tries=150
        )

        if matching2 > matching:
            return matching2, msg_A2, msg_B2, hash_A2, hash_B2
        return matching, msg_A, msg_B, hash_A, hash_B


# ============================================================
# EVOLUTIONARY BACKDOOR DISCOVERER
# ============================================================

class BackdoorDiscoveryEngine:
    """
    Evolutionary algorithm to discover backdoor conditions.

    Uses:
    1. Population of potential backdoors (genomes)
    2. Fitness = hash matching bits under condition
    3. Selection, crossover, mutation
    4. Elitism to keep best solutions
    5. Diversity maintenance
    """

    def __init__(self, population_size=50, num_rounds=64):
        self.population_size = population_size
        self.num_rounds = num_rounds
        self.population = []
        self.generation = 0
        self.best_ever = None
        self.history = []
        self.optimizer = SATConditionOptimizer(num_rounds)

        # Statistics
        self.stats = {
            'generations': 0,
            'evaluations': 0,
            'best_fitness': 0,
            'best_matching': 0,
            'collisions_found': 0
        }

    def initialize_population(self):
        """Create initial diverse population"""
        print("\n  Initializing population...")
        self.population = []

        strategies = [
            # Very sparse differential (1 bit)
            lambda g: (g.randomize(0.01, 0.002, 0), "sparse_diff"),
            # Dense message constraints
            lambda g: (g.randomize(0.15, 0.002, 0), "dense_msg"),
            # With correlations
            lambda g: (g.randomize(0.05, 0.002, 5), "correlated"),
            # Word-aligned constraints
            lambda g: self._word_aligned_init(g),
            # K-constant inspired
            lambda g: self._k_constant_init(g),
            # Symmetric pattern
            lambda g: self._symmetric_init(g),
            # Low hamming weight
            lambda g: self._low_hamming_init(g),
            # Random
            lambda g: (g.randomize(0.08, 0.003, 2), "random"),
        ]

        genomes_per_strategy = self.population_size // len(strategies)

        for strategy in strategies:
            for _ in range(genomes_per_strategy):
                genome = BackdoorGenome()
                strategy(genome)
                self.population.append(genome)

        # Fill remaining with random
        while len(self.population) < self.population_size:
            genome = BackdoorGenome()
            genome.randomize(random.uniform(0.02, 0.15),
                           random.uniform(0.001, 0.01),
                           random.randint(0, 5))
            self.population.append(genome)

        print(f"  Created {len(self.population)} initial genomes")

    def _word_aligned_init(self, genome):
        """Initialize with word-aligned constraints"""
        # Constrain specific words entirely
        words_to_constrain = random.sample(range(16), random.randint(1, 4))
        for word in words_to_constrain:
            val = random.choice([0, 1])
            for bit in range(32):
                genome.msg_constraints[word * 32 + bit] = val + 1

        # Differential in one word
        diff_word = random.randint(0, 15)
        diff_bit = random.randint(0, 31)
        genome.diff_constraints[diff_word * 32 + diff_bit] = 2

        return genome, "word_aligned"

    def _k_constant_init(self, genome):
        """Initialize based on K constants pattern"""
        # Use bits from K constants as constraints
        k_idx = random.randint(0, 63)
        k_val = K_CONSTANTS[k_idx]

        word_idx = random.randint(0, 15)
        for bit in range(32):
            if random.random() < 0.3:
                genome.msg_constraints[word_idx * 32 + bit] = ((k_val >> bit) & 1) + 1

        genome.diff_constraints[random.randint(0, 511)] = 2

        return genome, "k_constant"

    def _symmetric_init(self, genome):
        """Initialize with symmetric message pattern"""
        # Force M[i] related to M[15-i]
        for i in range(8):
            for bit in range(32):
                if random.random() < 0.1:
                    val = random.choice([1, 2])
                    genome.msg_constraints[i * 32 + bit] = val
                    genome.msg_constraints[(15 - i) * 32 + bit] = val
                    genome.correlations.append((i * 32 + bit, (15 - i) * 32 + bit, 0))

        genome.diff_constraints[random.randint(0, 255)] = 2

        return genome, "symmetric"

    def _low_hamming_init(self, genome):
        """Initialize with low Hamming weight constraint"""
        # Force many bits to 0
        for i in range(512):
            if random.random() < 0.2:
                genome.msg_constraints[i] = 1  # Force to 0

        genome.diff_constraints[random.randint(0, 511)] = 2

        return genome, "low_hamming"

    def evaluate_genome(self, genome):
        """Evaluate fitness of a genome"""
        matching, msg_A, msg_B, hash_A, hash_B = self.optimizer.optimize(genome)

        genome.matching_bits = matching
        genome.msg_A = msg_A
        genome.msg_B = msg_B
        genome.hash_A = hash_A
        genome.hash_B = hash_B
        genome.is_collision = (matching == 256)

        # Fitness function: prioritize matching bits, penalize complexity slightly
        constraint_count = np.sum(genome.msg_constraints != 0)
        complexity_penalty = 0.001 * constraint_count

        genome.fitness = matching - complexity_penalty

        self.stats['evaluations'] += 1

        return genome

    def evaluate_population(self):
        """Evaluate all genomes in parallel"""
        # Use thread pool for parallel evaluation
        with ThreadPoolExecutor(max_workers=8) as executor:
            futures = [executor.submit(self.evaluate_genome, g) for g in self.population]
            self.population = [f.result() for f in futures]

    def select_parents(self, tournament_size=5):
        """Tournament selection"""
        parents = []
        for _ in range(self.population_size):
            tournament = random.sample(self.population, min(tournament_size, len(self.population)))
            winner = max(tournament, key=lambda g: g.fitness)
            parents.append(winner)
        return parents

    def create_next_generation(self):
        """Create next generation through selection, crossover, mutation"""
        # Sort by fitness
        self.population.sort(key=lambda g: g.fitness, reverse=True)

        # Elitism: keep top 10%
        elite_count = max(2, self.population_size // 10)
        new_population = self.population[:elite_count]

        # Select parents
        parents = self.select_parents()

        # Create offspring
        while len(new_population) < self.population_size:
            if random.random() < 0.7:  # Crossover
                p1, p2 = random.sample(parents, 2)
                child = p1.crossover(p2, self.generation + 1)
            else:  # Mutation only
                parent = random.choice(parents)
                child = parent.mutate(0.1, self.generation + 1)

            # Always mutate a bit
            if random.random() < 0.3:
                child = child.mutate(0.05, self.generation + 1)

            new_population.append(child)

        self.population = new_population[:self.population_size]
        self.generation += 1

    def inject_diversity(self):
        """Inject fresh random genomes to maintain diversity"""
        # Replace worst 20% with fresh random
        self.population.sort(key=lambda g: g.fitness, reverse=True)
        inject_count = self.population_size // 5

        for i in range(inject_count):
            genome = BackdoorGenome()
            genome.randomize(random.uniform(0.02, 0.12),
                           random.uniform(0.002, 0.008),
                           random.randint(0, 3))
            self.population[-(i+1)] = genome

    def run(self, max_generations=100, target_bits=256, verbose=True):
        """Run evolutionary search"""
        print("\n" + "="*70)
        print("  EVOLUTIONARY BACKDOOR DISCOVERY")
        print("="*70)

        self.initialize_population()

        start_time = time.time()
        no_improvement_count = 0
        last_best = 0

        for gen in range(max_generations):
            self.generation = gen

            # Evaluate
            self.evaluate_population()

            # Track best
            current_best = max(self.population, key=lambda g: g.matching_bits)

            if self.best_ever is None or current_best.matching_bits > self.best_ever.matching_bits:
                self.best_ever = BackdoorGenome()
                self.best_ever.msg_constraints = current_best.msg_constraints.copy()
                self.best_ever.diff_constraints = current_best.diff_constraints.copy()
                self.best_ever.correlations = current_best.correlations.copy()
                self.best_ever.matching_bits = current_best.matching_bits
                self.best_ever.fitness = current_best.fitness
                self.best_ever.msg_A = current_best.msg_A
                self.best_ever.msg_B = current_best.msg_B
                self.best_ever.hash_A = current_best.hash_A
                self.best_ever.hash_B = current_best.hash_B
                self.best_ever.is_collision = current_best.is_collision

                no_improvement_count = 0
            else:
                no_improvement_count += 1

            # Statistics
            avg_matching = np.mean([g.matching_bits for g in self.population])
            max_matching = max(g.matching_bits for g in self.population)

            self.history.append({
                'generation': gen,
                'avg_matching': avg_matching,
                'max_matching': max_matching,
                'best_ever': self.best_ever.matching_bits
            })

            if verbose and gen % 5 == 0:
                elapsed = time.time() - start_time
                summary = current_best.get_constraint_summary()
                print(f"\n  Gen {gen:3d} | Best: {max_matching:3d}/256 | "
                      f"Avg: {avg_matching:.1f} | Ever: {self.best_ever.matching_bits:3d} | "
                      f"Time: {elapsed:.1f}s")
                print(f"          | Constraints: {summary['total_msg_constraints']} msg, "
                      f"{summary['diff_bits']} diff, {summary['correlations']} corr")

            # Check for collision
            if current_best.is_collision:
                print("\n" + "🚨"*30)
                print("  BACKDOOR FOUND! FULL COLLISION!")
                print("🚨"*30)
                self.stats['collisions_found'] += 1
                break

            # Check if we hit target
            if current_best.matching_bits >= target_bits:
                print(f"\n  Target {target_bits} bits reached!")
                break

            # Diversity injection if stuck
            if no_improvement_count >= 15:
                if verbose:
                    print(f"  [Injecting diversity - stuck for {no_improvement_count} generations]")
                self.inject_diversity()
                no_improvement_count = 0

            # Create next generation
            self.create_next_generation()

        self.stats['generations'] = self.generation
        self.stats['best_matching'] = self.best_ever.matching_bits if self.best_ever else 0

        return self.best_ever

    def analyze_best(self):
        """Detailed analysis of best found condition"""
        if not self.best_ever:
            print("No best genome found yet")
            return

        print("\n" + "="*70)
        print("  BEST DISCOVERED CONDITION ANALYSIS")
        print("="*70)

        genome = self.best_ever
        summary = genome.get_constraint_summary()

        print(f"\n  Matching bits: {genome.matching_bits}/256")
        print(f"  Is collision: {genome.is_collision}")

        print(f"\n  MESSAGE CONSTRAINTS:")
        print(f"    Bits forced to 0: {summary['msg_zeros']}")
        print(f"    Bits forced to 1: {summary['msg_ones']}")
        print(f"    Total constrained: {summary['total_msg_constraints']}/512")

        print(f"\n  DIFFERENTIAL:")
        print(f"    Active diff bits: {summary['diff_bits']}")

        # Show which bits are constrained
        print(f"\n  CONSTRAINT PATTERN:")
        for word in range(16):
            constrained = []
            for bit in range(32):
                idx = word * 32 + bit
                if genome.msg_constraints[idx] == 1:
                    constrained.append(f"{bit}=0")
                elif genome.msg_constraints[idx] == 2:
                    constrained.append(f"{bit}=1")
            if constrained:
                print(f"    Word {word:2d}: {', '.join(constrained[:10])}{'...' if len(constrained) > 10 else ''}")

        print(f"\n  DIFFERENTIAL BITS:")
        diff_bits = []
        for i in range(512):
            if genome.diff_constraints[i] == 2:
                word = i // 32
                bit = i % 32
                diff_bits.append(f"W{word}[{bit}]")
        print(f"    {', '.join(diff_bits)}")

        if genome.correlations:
            print(f"\n  CORRELATIONS:")
            for (i, j, rel) in genome.correlations[:10]:
                rel_str = "==" if rel == 0 else "!="
                print(f"    bit[{i}] {rel_str} bit[{j}]")

        print(f"\n  MESSAGES:")
        if genome.msg_A:
            print(f"    A: {genome.msg_A.hex()}")
        if genome.msg_B:
            print(f"    B: {genome.msg_B.hex()}")

        print(f"\n  HASHES:")
        if genome.hash_A:
            print(f"    H(A): {genome.hash_A}")
        if genome.hash_B:
            print(f"    H(B): {genome.hash_B}")


# ============================================================
# PATTERN MINING
# ============================================================

class PatternMiner:
    """
    Analyzes successful genomes to find common patterns.
    These patterns might indicate structural weaknesses.
    """

    def __init__(self):
        self.patterns = defaultdict(int)
        self.successful_genomes = []

    def add_genome(self, genome, threshold=150):
        """Add genome if it's successful enough"""
        if genome.matching_bits >= threshold:
            self.successful_genomes.append(genome)
            self._extract_patterns(genome)

    def _extract_patterns(self, genome):
        """Extract patterns from a genome"""
        # Word-level patterns
        for word in range(16):
            zeros = sum(1 for b in range(32) if genome.msg_constraints[word*32+b] == 1)
            ones = sum(1 for b in range(32) if genome.msg_constraints[word*32+b] == 2)

            if zeros > 16:
                self.patterns[f"word_{word}_mostly_zeros"] += 1
            if ones > 16:
                self.patterns[f"word_{word}_mostly_ones"] += 1

        # Differential patterns
        diff_words = set()
        for i in range(512):
            if genome.diff_constraints[i] == 2:
                diff_words.add(i // 32)

        if len(diff_words) == 1:
            self.patterns["single_word_diff"] += 1

        # Bit position patterns
        for i in range(512):
            if genome.msg_constraints[i] != 0:
                self.patterns[f"constrained_bit_{i}"] += 1

    def report(self):
        """Report discovered patterns"""
        print("\n" + "="*70)
        print("  PATTERN MINING RESULTS")
        print("="*70)

        if not self.successful_genomes:
            print("  No successful genomes to analyze")
            return

        print(f"\n  Analyzed {len(self.successful_genomes)} successful genomes")

        # Top patterns
        sorted_patterns = sorted(self.patterns.items(), key=lambda x: -x[1])

        print(f"\n  TOP RECURRING PATTERNS:")
        for pattern, count in sorted_patterns[:20]:
            pct = 100 * count / len(self.successful_genomes)
            print(f"    {pattern}: {count} ({pct:.1f}%)")

        # Common constraint positions
        constraint_counts = np.zeros(512)
        for genome in self.successful_genomes:
            constraint_counts += (genome.msg_constraints != 0).astype(int)

        hot_bits = np.argsort(constraint_counts)[-20:][::-1]
        print(f"\n  MOST COMMONLY CONSTRAINED BITS:")
        for bit in hot_bits:
            if constraint_counts[bit] > 0:
                word = bit // 32
                b = bit % 32
                print(f"    Word {word}, Bit {b}: constrained in {int(constraint_counts[bit])} genomes")


# ============================================================
# GRADIENT-LIKE LOCAL SEARCH
# ============================================================

class LocalSearchOptimizer:
    """
    Local search to refine a genome.
    Tries small changes and keeps improvements.
    """

    def __init__(self):
        pass

    def optimize(self, genome, max_iterations=1000, verbose=False):
        """Local search optimization"""
        best = genome
        best_matching = genome.matching_bits

        for iteration in range(max_iterations):
            # Try random small change
            candidate = BackdoorGenome()
            candidate.msg_constraints = best.msg_constraints.copy()
            candidate.diff_constraints = best.diff_constraints.copy()
            candidate.correlations = best.correlations.copy()

            # Small mutation
            change_type = random.choice(['flip_msg', 'add_msg', 'remove_msg',
                                        'move_diff', 'add_corr', 'remove_corr'])

            if change_type == 'flip_msg':
                idx = random.randint(0, 511)
                if candidate.msg_constraints[idx] != 0:
                    candidate.msg_constraints[idx] = 3 - candidate.msg_constraints[idx]

            elif change_type == 'add_msg':
                idx = random.randint(0, 511)
                if candidate.msg_constraints[idx] == 0:
                    candidate.msg_constraints[idx] = random.choice([1, 2])

            elif change_type == 'remove_msg':
                constrained = np.where(candidate.msg_constraints != 0)[0]
                if len(constrained) > 0:
                    idx = random.choice(constrained)
                    candidate.msg_constraints[idx] = 0

            elif change_type == 'move_diff':
                # Move differential to adjacent bit
                diff_bits = np.where(candidate.diff_constraints == 2)[0]
                if len(diff_bits) > 0:
                    old_idx = random.choice(diff_bits)
                    new_idx = old_idx + random.choice([-1, 1])
                    if 0 <= new_idx < 512:
                        candidate.diff_constraints[old_idx] = 0
                        candidate.diff_constraints[new_idx] = 2

            elif change_type == 'add_corr':
                i = random.randint(0, 511)
                j = random.randint(0, 511)
                if i != j:
                    candidate.correlations.append((i, j, random.choice([0, 1])))

            elif change_type == 'remove_corr':
                if candidate.correlations:
                    candidate.correlations.pop(random.randint(0, len(candidate.correlations)-1))

            # Evaluate
            matching, msg_A, msg_B, hash_A, hash_B = sha256_differential_test(
                candidate.msg_constraints,
                candidate.diff_constraints,
                candidate.correlations,
                num_tries=20
            )

            # Keep if better
            if matching > best_matching:
                best = candidate
                best.matching_bits = matching
                best.msg_A = msg_A
                best.msg_B = msg_B
                best.hash_A = hash_A
                best.hash_B = hash_B
                best_matching = matching

                if verbose:
                    print(f"    Iteration {iteration}: improved to {matching}/256")

                if matching == 256:
                    break

        return best


# ============================================================
# MAIN DISCOVERY ENGINE
# ============================================================

class BackdoorHunter:
    """Main interface for backdoor discovery"""

    def __init__(self):
        self.discovery_engine = None
        self.local_optimizer = LocalSearchOptimizer()
        self.pattern_miner = PatternMiner()
        self.all_discoveries = []

    def run_discovery(self, generations=100, population=50, runs=5):
        """Run multiple discovery attempts"""
        print("\n" + "#"*70)
        print("#" + " "*15 + "SHA-256 BACKDOOR DISCOVERY" + " "*21 + "#")
        print("#" + " "*12 + "Evolutionary Automatic Search" + " "*21 + "#")
        print("#"*70)

        best_overall = None

        for run in range(runs):
            print(f"\n\n{'='*70}")
            print(f"  DISCOVERY RUN {run + 1}/{runs}")
            print(f"{'='*70}")

            # Run evolution
            engine = BackdoorDiscoveryEngine(population_size=population)
            best = engine.run(max_generations=generations, verbose=True)

            if best:
                self.pattern_miner.add_genome(best)
                self.all_discoveries.append(best)

                # Local search refinement
                print("\n  Refining with local search...")
                refined = self.local_optimizer.optimize(best, max_iterations=500, verbose=False)

                if refined.matching_bits > best.matching_bits:
                    print(f"  Local search improved: {best.matching_bits} → {refined.matching_bits}")
                    best = refined
                    self.pattern_miner.add_genome(refined)

                if best_overall is None or best.matching_bits > best_overall.matching_bits:
                    best_overall = best

                # Check for collision
                if best.is_collision:
                    self._report_collision(best)
                    return best

        # Final analysis
        self._final_report(best_overall)

        return best_overall

    def _report_collision(self, genome):
        """Report a found collision"""
        print("\n\n" + "🚨"*35)
        print("  CRITICAL: BACKDOOR/COLLISION DISCOVERED!")
        print("🚨"*35)

        print(f"\n  This condition creates a SHA-256 collision:")

        # Show the condition
        summary = genome.get_constraint_summary()
        print(f"\n  BACKDOOR CONDITION:")
        print(f"    Message bits forced to 0: {summary['msg_zeros']}")
        print(f"    Message bits forced to 1: {summary['msg_ones']}")
        print(f"    Differential bits: {summary['diff_bits']}")

        print(f"\n  COLLIDING MESSAGES:")
        print(f"    A: {genome.msg_A.hex()}")
        print(f"    B: {genome.msg_B.hex()}")

        print(f"\n  SHARED HASH:")
        print(f"    {genome.hash_A}")

        # Save to file
        with open("sha256_backdoor_found.json", "w") as f:
            json.dump(genome.to_dict(), f, indent=2)
        print(f"\n  Saved to: sha256_backdoor_found.json")

    def _final_report(self, best):
        """Final report of discovery run"""
        print("\n\n" + "="*70)
        print("  DISCOVERY COMPLETE")
        print("="*70)

        if not best:
            print("  No conditions found")
            return

        print(f"\n  Best matching bits achieved: {best.matching_bits}/256")
        print(f"  Collision found: {best.is_collision}")

        # Analyze best
        if best.matching_bits >= 128:
            print(f"\n  ⚠️  NOTABLE: Achieved {best.matching_bits}/256 matching bits")
            print("  This exceeds random chance (128 expected)")

            summary = best.get_constraint_summary()
            print(f"\n  CONDITION THAT ACHIEVED THIS:")
            print(f"    Total message constraints: {summary['total_msg_constraints']}")
            print(f"    Differential bits: {summary['diff_bits']}")

            print(f"\n  MESSAGES:")
            if best.msg_A:
                print(f"    A: {best.msg_A.hex()}")
            if best.msg_B:
                print(f"    B: {best.msg_B.hex()}")

            print(f"\n  HASHES:")
            if best.hash_A:
                print(f"    H(A): {best.hash_A}")
            if best.hash_B:
                print(f"    H(B): {best.hash_B}")

        # Pattern analysis
        self.pattern_miner.report()

        # Save all discoveries
        discoveries_data = [g.to_dict() for g in self.all_discoveries]
        with open("sha256_discoveries.json", "w") as f:
            json.dump({
                'best_matching': best.matching_bits,
                'discoveries': discoveries_data
            }, f, indent=2)
        print(f"\n  All discoveries saved to: sha256_discoveries.json")

        # Verdict
        print("\n" + "-"*70)
        if best.is_collision:
            print("  🚨 VERDICT: BACKDOOR FOUND")
        elif best.matching_bits >= 200:
            print("  ⚠️  VERDICT: SUSPICIOUS WEAKNESS DETECTED")
            print("     Further investigation recommended")
        elif best.matching_bits >= 160:
            print("  ⚡ VERDICT: MINOR ANOMALY")
            print("     Slightly above random, likely statistical noise")
        else:
            print("  ✓ VERDICT: NO BACKDOOR DETECTED")
            print("     SHA-256 appears resistant to discovered conditions")


# ============================================================
# INTERACTIVE MENU
# ============================================================

def main():
    print("\n" + "="*70)
    print("  🔬 SHA-256 BACKDOOR DISCOVERY ENGINE")
    print("  Automatic Search for Hidden Weaknesses")
    print("="*70)

    print("""
PURPOSE:
========
Unlike predefined tests, this tool DISCOVERS backdoor conditions
automatically using evolutionary algorithms.

HOW IT WORKS:
=============
1. Generate random "backdoor conditions" (constraints on messages)
2. Test each condition: does it make collisions easier?
3. Evolve conditions that perform better
4. Refine with local search
5. Analyze patterns in successful conditions

WHAT IT FINDS:
==============
• Condition C on message A such that:
  ∃B ≠ A: SHA256(A) ≈ SHA256(B) when C(A) is true

If matching bits significantly exceed 128 (random baseline),
the condition represents a structural weakness.

If matching bits reach 256, it's a COLLISION = BACKDOOR.
""")

    hunter = BackdoorHunter()

    while True:
        print("\n" + "-"*50)
        print("OPTIONS:")
        print("  1. Quick discovery (50 generations, 3 runs)")
        print("  2. Standard discovery (100 generations, 5 runs)")
        print("  3. Deep discovery (200 generations, 10 runs)")
        print("  4. Custom discovery")
        print("  5. View pattern analysis")
        print("  q. Quit")

        choice = input("\nChoice: ").strip().lower()

        if choice == 'q':
            print("Goodbye!")
            break

        elif choice == '1':
            hunter.run_discovery(generations=50, population=30, runs=3)

        elif choice == '2':
            hunter.run_discovery(generations=100, population=50, runs=5)

        elif choice == '3':
            hunter.run_discovery(generations=200, population=80, runs=10)

        elif choice == '4':
            try:
                gens = int(input("Generations per run (default 100): ").strip() or "100")
                pop = int(input("Population size (default 50): ").strip() or "50")
                runs = int(input("Number of runs (default 5): ").strip() or "5")
                hunter.run_discovery(generations=gens, population=pop, runs=runs)
            except ValueError:
                print("Invalid input")

        elif choice == '5':
            hunter.pattern_miner.report()

        else:
            print("Invalid choice")


if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 36.3 MB/s eta 0:00:00
  🔬 SHA-256 BACKDOOR DISCOVERY ENGINE
  Automatic Search for Hidden Weaknesses

  🔬 SHA-256 BACKDOOR DISCOVERY ENGINE
  Automatic Search for Hidden Weaknesses

PURPOSE:
Unlike predefined tests, this tool DISCOVERS backdoor conditions
automatically using evolutionary algorithms.

HOW IT WORKS:
1. Generate random "backdoor conditions" (constraints on messages)
2. Test each condition: does it make collisions easier?
3. Evolve conditions that perform better
4. Refine with local search
5. Analyze patterns in successful conditions

WHAT IT FINDS:
• Condition C on message A such that:
  ∃B ≠ A: SHA256(A) ≈ SHA256(B) when C(A) is true

If matching bits significantly exceed 128 (random baseline),
the condition represents a structural weakness.

If matching bits reach 256, it's a COLLISION = BACKDOOR.


--------------------------------------------------
OPTIONS:
  1. Quick discovery (50 generations, 3 runs)
  2. Standar